In [ ]:
#piplite presence will be shown with these statememnts
try:    
    import piplite
except ModuleNotFoundError:
    piplite=None
# Use these packages for download of data
if piplite:
    await piplite.install("mplfinance")
    await piplite.install("finta")
    await piplite.install("ipympl")
    await piplite.install("ipywidgets")
    try:
        await piplite.install("/pypi/backtrader-1.9.76.123-py3-none-any.whl")
    except ValueError as e:
        print("localExampleEnv-failed")
        try: 
            await piplite.install("http://localhost/staticcollected/_output/pypi/backtrader-1.9.76.123-py3-none-any.whl")
        except ValueError as e:
            print("localProd00env-failed")
            await piplite.install("http://www.stockwhiz.in/staticcollected/_output/pypi/backtrader-1.9.76.123-py3-none-any.whl")
    # This is for the fetch in piplite
    import js 
    from pyodide.ffi import to_js
    from js import Object
else:
    import yfinance
#3 Imports of all the files
import json
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import mplfinance as mpf
import scipy as sp
from IPython.display import display, Markdown
import backtrader as bt
import finta
import ipywidgets as widgets

%matplotlib widget
pd.options.display.max_columns = None

---

# 09 Bullish Engulfing backtest

Click $>>$ button to run the entire script first

**Buy Signal**
> BUY 100 shares when a Bullish Engulfing pattern occurs at 10min candle interval using CandleStick chart. Enter trade between 09:00 to 23:59 Enter trade between 09:00 to 23:59

**Sell Signal**
> SELL 100 shares when a Bearish Engulfing patter occurs at 10min candle interval using CandleStick chart. or at stop loss % of 1.0 or target profit % of 2.0 at 15min candle interval using Candlestick chart chart.

Please feel free to change to any **NSE SYMBOL** to the below list, separated by a comma

In [ ]:
symbols="SBILIFE"

In [ ]:
corsprox = "https://corsproxy.io/?"
apiBaseURL = "https://query1.finance.yahoo.com/v8/finance/chart/"
rangeOfData = "7d"
intervalOfData = "10m"
apiSymbol = []
for symbol in symbols.split(","):
    apiSymbol.append(symbol+".NS")
stockPandasTot = {}
for apiSymbolIndivi in apiSymbol:
    print(apiSymbolIndivi)
    apiCompleteURL = corsprox+apiBaseURL+apiSymbolIndivi+\
        "?range="+rangeOfData+"&interval="+intervalOfData
    options = { "mode": "cors",
                "credentials":"omit",
                "headers": {'Accept': 'application/json',
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
                            'Origin':'https://yahoo.com',
                            }
    }
    if piplite:
        resp = await js.fetch(apiCompleteURL, to_js(options, dict_converter=Object.fromEntries))
        text = await resp.text()
        pan = pd.read_json(text)
        a_hist = pan['chart'][1][0]
        stockPandas = pd.DataFrame(a_hist['indicators']['quote'][0],
                               index=pd.to_datetime(a_hist['timestamp'],unit='s'))
        stockPandas = stockPandas.reindex(columns=['open','high','low','close','volume'])
        stockPandas = stockPandas[::-1]
    else:
        stockPandas = yfinance.download(apiSymbolIndivi,period=rangeOfData,interval=intervalOfData,progress=False)
        stockPandas = stockPandas.round(2).rename(columns={
                                                    'Open': 'open',
                                                    'High': 'high',
                                                    'Low': 'low',
                                                    'Close': 'close',
                                                    'Adj Close': 'adj close',
                                                    'Volume': 'volume'
                                                })
        stockPandas = stockPandas[::-1]
    stockPandasTot[apiSymbolIndivi] = stockPandas

## A Bullish Engulfing pattern trade

<img src="https://scanz.com/wp-content/uploads/2018/12/bollingerbands.jpg" width="900" height="300"/>

### Buy Signal
> BUY 100 shares when a Bullish Engulfing pattern occurs at 10min candle interval using CandleStick chart. Enter trade between 09:00 to 23:59 Enter trade between 09:00 to 23:59

- Close > Open
- Shadows should be within $0.3\%$
- Candle should be from $1\%$ to $10\%$ range

### Sell Signal
> SELL 100 shares when a Bearish Engulfing patter occurs at 10min candle interval using CandleStick chart. or at stop loss % of 1.0 or target profit % of 2.0 at 15min candle interval using Candlestick chart chart.

Target signal achieved if 
- Open > Close
- Shadows should be within $0.3\%$
- Candle should be from $1\%$ to $10\%$ range

If either of the below conditions are met
- Stop Loss condition met if $close[0] < order.buy*0.99$
- Target price condition met if $close[0] > order.buy*1.01$

In [ ]:
display(Markdown("# Set the variables to this strategy here"))
display(Markdown("> We will improve UI shortly"))
display(Markdown("If interested on the `coding` aspect, please click the 3 dot's above to see comments on code and change accordingly"))
@widgets.interact(stopLoss=widgets.FloatSlider(
                                    value=1,
                                    min=0,
                                    max=5,
                                    step=0.01,
                                    description='Stop Loss %:',
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='.2f',
                                ),\
                  targetPrice=widgets.FloatSlider(
                                    value=2,
                                    min=0,
                                    max=3,
                                    step=0.01,
                                    description='Target Price %:',
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='.2f',
                                ),\
                  cash=widgets.IntSlider(
                                    value=50000,
                                    min=0,
                                    max=100000,
                                    step=500,
                                    description='Cash (Rs):',
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                ),\
                  BbPeriod=widgets.IntSlider(
                                    value=20,
                                    min=5,
                                    max=30,
                                    step=1,
                                    description='BB period (candles):',
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                ),\
                  size=widgets.IntSlider(
                                    value=10,
                                    min=1,
                                    max=100,
                                    step=1,
                                    description='Size (#):',
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                ),\
                 )
def voilas(cash,stopLoss,targetPrice,BbPeriod,size):
    class BullishMACrossover(bt.Strategy):
        def __init__(self):
            #Some indicators
            self.bb = bt.indicators.BollingerBands(period=BbPeriod,devfactor=1)
            #if close > bb then signal=+1
            self.signalSell = bt.indicators.CrossOver(self.datas[0].close,self.bb.top,plot=False)
            self.signalBuy = bt.indicators.CrossOver(self.bb.bot,self.datas[0].close,plot=False)
            # To keep track of pending orders and buy price/commission
            self.order = None
            self.buyprice = None
            self.buycomm = None

        def log(self, txt, dt=None):
            ''' Logging function for this strategy'''
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s | %s' % (dt.isoformat(),self.datas[0].datetime.time(0), txt))

        def notify_order(self, order):
            if order.status in [order.Submitted, order.Accepted]:
                # Buy/Sell order submitted/accepted to/by broker - Nothing to do
                return

            # Check if an order has been completed
            # Attention: broker could reject order if not enough cash
            if order.status in [order.Completed]:
                if order.isbuy():
                    self.log(
                        'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm))

                    self.buyprice = order.executed.price
                    self.buycomm = order.executed.comm
                else:  # Sell
                    return
                    self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                             (order.executed.price,
                              order.executed.value,
                              order.executed.comm))

                self.bar_executed = len(self)

            elif order.status in [order.Canceled, order.Margin, order.Rejected]:
                self.log('Order Canceled/Margin/Rejected')

            self.order = None

        def notify_trade(self, trade):
            if not trade.isclosed:
                return

            self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                     (trade.pnl, trade.pnlcomm))

        def next(self):
            if not self.position:
                if self.signalSell[0] > 0:
                    self.order = self.sell(size=size)
            else:
                if self.signalBuy[0] > 0:
                    self.order = self.buy(size=size)
                    self.log("target signal")
                if self.datas[0].close[0] < (1-targetPrice/100)*self.position.price:
                    self.log("target price")
                    self.order = self.buy(size=size)
                if self.datas[0].close[0] > (1+stopLoss/100)*self.position.price:
                    self.log("Stop loss")
                    self.order = self.buy(size=size)
                    
    cerebro = bt.Cerebro()

    data = bt.feeds.PandasData(dataname=stockPandasTot[symbol+".NS"][::-1],timeframe=bt.TimeFrame.Days)
    cerebro.adddata(data)
    cerebro.addstrategy(BullishMACrossover)

    cerebro.broker.setcash(cash)

    cerebro.broker.setcommission(commission=0.001)
    cerebro.broker.set_shortcash(True)
    cerebro.broker.set_coc(True)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.plot(style="candlestick")

---
### Understanding above plot

> Please increase the size of the plot to the entire width of the screen, before reading this section. You can do the same by dragging the triangular element shown at the bottom right corner of the plot

The plot above has quite a lines and legends on them.
The legend is available on the left side indicating various items.

We shall move from the bottom most element and move upwards.
#### Bollinger Bands
The Bollinger bands provide of volatility, oversold and even overbought prices. It contains 3 lines namely the top, middle and bottom lines.
[https://en.wikipedia.org/wiki/Bollinger_Bands]
#### Volume
The volume is indicated by bars of either gray or red colour. The respective value is availabe on the left part of the plot along the y-axis. Exact values for volume cannot be extracted currently. A red bar indicates a decrease in stock price over the time period. A grey bar indicates an increase in stock price over the given time period.
#### Triangles
The triangles represent positions taken up according to the strategy mentioned by code above. A buy position/ signal will be generated if the triangle is green and pointed upwards, while a sell position/ sell signal is generated with the red downward pointing triangle.
#### Trading dots
The next canvas involves a set of dots representing either profit or losses which have occured by following the strategy. Here too the magnitudes can be separated by the colour of the dot and the cursor value on hovering over dots. One can also approximate the values with the data on the scale located on the right.
#### Portfolio value
The final screen on the top indicates the portfolio value and cash value. Each are differentiated by the respective color.